In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import warnings # to ignore warnings
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [4]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [5]:
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
#     ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
#     js['q_idx'] = ndx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [7]:
# training data
input_file_path = '../SQuAD/translate-train/squad.translate.train.en-te.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [8]:
train.head()

,index,question,context,answer_start,text,c_id
0,57269b79dd62a815002e8aea,ఈ సమయంలో యూరప్‌లో ఏ చర్చి విస్తృతంగా వ్యాపించి...,కాథలిక్ చర్చి ఈ సమయంలో యూరప్ అంతటా వ్యాపించింద...,0,కాథలిక్ చర్చి,0
1,56e075f87aa994140058e509,బ్రీతి వాయిస్ అంటే ఏమిటి?,స్వర ఆకాంక్ష హల్లులు అని పిలవబడేవి బ్రీతి వాయి...,85,ఒక రకమైన ఫోనేషన్ లేదా స్వర మడతల వైబ్రేషన్,1
2,5726866b5951b619008f758f,"""నా"" కోసం డచ్ పదం ఏమిటి?",డచ్ వ్రాసిన వ్యాకరణం గత 100 సంవత్సరాలుగా సరళీక...,158,మిజ్న్,2
3,56d104d117492d1400aab73f,ప్రతి సంవత్సరం ఎంత మంది హెచ్‌హెచ్‌సి అత్యవసర గ...,"ప్రతి సంవత్సరం HHC యొక్క సౌకర్యాలు సుమారు 225,...",61,ఒక మిలియన్,3
4,5726627bf1498d1400e8ddd4,ఏ అన్వేషకుడు కాబోట్ మాదిరిగానే తప్పు చేసాడు?,ఇంగ్లాండ్ మరియు స్కాట్లాండ్ ప్రత్యేక రాజ్యాలు ...,460,క్రిస్టోఫర్ కొలంబస్,4


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87091 entries, 0 to 87090
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         87091 non-null  object
 1   question      87091 non-null  object
 2   context       87091 non-null  object
 3   answer_start  87091 non-null  int64 
 4   text          87091 non-null  object
 5   c_id          87091 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


In [19]:
# loop over all files in the directory "SQuAD/translate-train/ using glob 
# to get all the files in the directory
from glob import glob

for f in glob("./SQuAD/translate-train/*.json"):
    # load the file in dataframe using the function squad_json_to_dataframe_train
    train_data = squad_json_to_dataframe_train(input_file_path=f,record_path=record_path,verbose=0)
    # print name of file and shape of the dataframe
    print(f,train_data.shape)

./SQuAD/translate-train/squad.translate.train.en-ar.json (86787, 6)
./SQuAD/translate-train/squad.translate.train.en-bn.json (85181, 6)
./SQuAD/translate-train/squad.translate.train.en-de.json (82603, 6)
./SQuAD/translate-train/squad.translate.train.en-el.json (79946, 6)
./SQuAD/translate-train/squad.translate.train.en-es.json (87488, 6)
./SQuAD/translate-train/squad.translate.train.en-fi.json (86914, 6)
./SQuAD/translate-train/squad.translate.train.en-hi.json (85804, 6)
./SQuAD/translate-train/squad.translate.train.en-id.json (87167, 6)
./SQuAD/translate-train/squad.translate.train.en-ko.json (85757, 6)
./SQuAD/translate-train/squad.translate.train.en-ru.json (84869, 6)
./SQuAD/translate-train/squad.translate.train.en-sw.json (86311, 6)
./SQuAD/translate-train/squad.translate.train.en-te.json (87091, 6)
./SQuAD/translate-train/squad.translate.train.en-th.json (85846, 6)
./SQuAD/translate-train/squad.translate.train.en-tr.json (86511, 6)
./SQuAD/translate-train/squad.translate.train.en